In [1]:
import numpy as np
import torch
import sys
import os
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/kidney/CosMx_kidney")


/home/luqiaolin/anaconda3/envs/baseline_code/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!ls
!pwd

annot.csv	 Run1080_SP19_1139   Run1081_SP17_2566	Run1087_SP19_4061
customlocs.csv	 Run1080_SP20_10838  Run1081_SP18_3323	Run1087_SP20_1098
Kidney_2566.pdf  Run1080_SP20_642    Run1087_SP17_8693	um.csv
raw.RDS		 Run1080_SP21_213    Run1087_SP18_8471
/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/kidney/CosMx_kidney


In [3]:
gene_expression = pd.read_csv('./Run1087_SP19_4061/Run1087_SP19_4061_exprMat_file.csv')
cell_boundary = pd.read_csv('./Run1087_SP19_4061/Run1087_SP19_4061_metadata_file.csv')
fov_position = pd.read_csv('./Run1087_SP19_4061/Run1087_SP19_4061_fov_positions_file.csv')
CosMx_cell_type = pd.read_csv('./annot.csv')

In [4]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]

In [5]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 21):  # 20
        for j in range(1, 11):  # 10
            spot_id = (i - 1) * 10 + j
            x = x_l + (5472 / 40.0) * 1 + (5472 / 20.0) * (i - 1)
            y = y_h - (3648 / 20.0) * 1 - (3648 / 10.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates


In [6]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        # spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)
        spot_id = 1 + 10 * int((center_x - 1e-6) / 273.6) + int((center_y - 1e-6) / 364.8)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    

In [7]:
fov_ids_lst = cell_boundary['fov'].unique()


In [8]:
data_final_result = pd.DataFrame(columns = [ 'fov', 'spot_id', 'cell_ID'])

fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_id)]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)


fov_ids_lst: [1 2 3 4 5 6 7 8]


100%|██████████| 8/8 [00:19<00:00,  2.46s/it]


In [9]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1, 201)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0


In [10]:
fov_spot_cells_stats = pd.DataFrame(columns = names)
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = list(range(1, 201))

for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)


In [11]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = (fov_expression.columns)[2:].tolist()
    
#     gene_count_dic = {}
#     for gene_name in genes_lst:
#         gene_count_dic[gene_name] = 0
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    
    

In [12]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = list(range(1, 201))
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = (gene_expression.columns)[2:].tolist()
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)

for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    print("fov_id:", fov_id)
#     print("fov_data", fov_data)
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
#     print("fov_gene_expression:", fov_gene_expression)
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)

  

fov_id: 1
fov_id: 2
fov_id: 3
fov_id: 4
fov_id: 5
fov_id: 6
fov_id: 7
fov_id: 8


In [13]:
SAMPLE_ID = 6

sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 11):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = []
print("cell:", sample_cell_count_dic)
print("fov:", sample_fov_count_dic)


sample_1_index_lst = []
cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
print("len of cell_id_lst:", len(cell_id_lst))
fov_lst = []
index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == SAMPLE_ID:
        sample_1_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    
    if fov_id not in sample_fov_count_dic[sample_id]:
        sample_fov_count_dic[sample_id].append(fov_id)
    
    if sample_id == SAMPLE_ID:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(sample_fov_count_dic)
print(fov_lst)
print(len(sample_1_index_lst))

len(sample_1_index_lst)

CosMx_cell_type_sample_1 = CosMx_cell_type.iloc[sample_1_index_lst]
CosMx_cell_type_sample_1.reset_index(drop=True, inplace=True)

for i in CosMx_cell_type_sample_1.index:
    CosMx_cell_type_sample_1.at[i,'cell_ID']= int(CosMx_cell_type_sample_1.iloc[i]["cell_ID"].split("_")[3])




cell: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
fov: {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}
len of cell_id_lst: 296838
{1: 28880, 2: 19616, 3: 28000, 4: 13455, 5: 61073, 6: 16474, 7: 12404, 8: 32534, 9: 34902, 10: 49500}
{1: [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 2: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 3: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 4: [18, 19, 20, 21, 22, 23, 24, 25], 5: [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 6: [1, 2, 3, 4, 5, 6, 7, 8], 7: [1, 2, 3, 4, 5, 6, 7], 8: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 9: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 10: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}
[1, 2, 3, 4, 5, 6, 7, 8]
16474


In [14]:
sample_1_dic = {}
for key in CosMx_cell_type_sample_1["cell_type"].tolist():
    if key not in sample_1_dic:
        sample_1_dic[key] = 1
    else:
        sample_1_dic[key] = sample_1_dic[key] + 1

In [15]:
cell_type_lst = set(CosMx_cell_type_sample_1['cell_type'].tolist())

In [16]:
spot_celld_mapping = data_final_result
cell_id_lst = CosMx_cell_type_sample_1["cell_ID"].tolist()

In [17]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
#     import ipdb
#     ipdb.set_trace()

    for cell_id in one_spot_cell_lst:
        
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic



In [18]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
fov_ids_lst

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [19]:
column_name_lst = ['fov', 'spot_id'] + sorted(cell_type_lst)
ground_truth_table = pd.DataFrame(columns = column_name_lst)


fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = list(range(1, 201))

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = (spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_1[(CosMx_cell_type_sample_1['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)


cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [20]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 21):  # 20
        for j in range(1, 11):  # 10
            spot_id = (i - 1) * 10 + j
            x = x_l + (5472 / 40.0) * 1 + (5472 / 20.0) * (i - 1)
            y = y_h - (3648 / 20.0) * 1 - (3648 / 10.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    return fov_spot_coordinates


In [21]:
fov_lst = fov_ids_lst
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,46564.577778,153487.822222
1,1.0,2.0,46564.577778,153123.022222
2,1.0,3.0,46564.577778,152758.222222
3,1.0,4.0,46564.577778,152393.422222
4,1.0,5.0,46564.577778,152028.622222
...,...,...,...,...
1595,8.0,196.0,32229.644444,178586.044444
1596,8.0,197.0,32229.644444,178221.244444
1597,8.0,198.0,32229.644444,177856.444444
1598,8.0,199.0,32229.644444,177491.644444


In [22]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]


file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_kidney/Run1087_SP19_4061.h5ad"

filtered_data.write_h5ad(file_path)



